In [2]:
from experta import *
#Get user input for price range, product type, and features
price_range = input("Enter the price range: ")
product_type = input("What are you looking for... ")
features = []

while True:
    feature = input(f"What features are you looking for in your {product_type}... enter 'done' when you are done: ")
    if feature.lower() == 'done' or len(feature) == 0: 
        break 
    features.append(feature)

#transforming price range to a discrete value
price_range = price_range.split(' ')
price_range = (int(price_range[0]) + int(price_range[2]))/2
if price_range < 1000:
    price_range2 = 'low'
elif price_range < 10000 and price_range >= 1000:
    price_range2 = 'medium'
elif price_range >= 10000:
    price_range2 = 'high'

    
print(price_range2, product_type, features)

features = ['waterproof']
class Recommender(KnowledgeEngine):
    #executes if the price range is high, the product type is a laptop, and the features include waterproof/ long battery life
    @Rule(Fact(price_range='high'), Fact(product_type='laptop'), Fact(features=MATCH.features))
    def recommend_laptop_high(self, features):
        if 'waterproof' in features:
            print("You should buy a Dell TSC-7010 Series")
        elif 'long battery life' in features:
            print("You should buy a MacBook Pro")
    
    #executes if the price range is high and the product type is a smartphone
    @Rule(Fact(price_range='high'), Fact(product_type='smartphone'), Fact(features=MATCH.features))
    def recommend_smartphone_high(self, features):
        if 'long battery life' in features and 'waterproof' in features:
            print("You should buy a Samsung Galaxy S20")
    #executes if the price range is low and the product type is a smartphone
    @Rule(Fact(price_range='low'), Fact(product_type='smartphone'))
    def recommend_smartphone_low(self):
        print("You should buy a Samsung Galaxy A10")
    #executes if the price range is medium and the product type is a smartphone
    @Rule(Fact(price_range='medium'), Fact(product_type='smartphone'))
    def recommend_smartphone_medium(self):
        print("You should buy a Samsung Galaxy A50")
    #executes if the price range is low and the product type is a jacket and the features include waterproof
    @Rule(Fact(price_range='low'), Fact(product_type='jacket'), Fact(features=MATCH.features))
    def recommend_jacket_low(self, features):
        if 'waterproof' in features:
            print("You should buy a North Face jacket")

    @Rule(Fact(price_range='high'), Fact(product_type='jacket'))
    def recommend_jacket_high(self):
        print("You should buy a Canada Goose jacket")

    @Rule(Fact(price_range='medium'), Fact(product_type='jacket'))
    def recommend_jacket_medium(self):
        print("You should buy an Outdoorer jacket")

# Creating a new engine
engine = Recommender()
# Resetting the engine
engine.reset()
# Adding facts to the engine
engine.declare(Fact(price_range=price_range), Fact(product_type=product_type), Fact(features=features))

# Running the engine
engine.run()

KeyboardInterrupt: Interrupted by user

In [ ]:
from experta import *

class Diagnoser(KnowledgeEngine):

    # Wildcard rule (executed only if no other rule has fired)
    @Rule(Fact(isue1=MATCH.isue1), Fact(isue2=MATCH.isue2), Fact(isue3=MATCH.isue3), NOT(Fact(diagnosed=True)))
    def noIssues(self):
        print('Could not diagnose the problem.')

    # Helper rule to mark diagnosed=True when any specific rule fires
    @Rule(OR(
        AND(Fact(isue1='slow'), Fact(isue2='noisy'), Fact(isue3='overheating')), 
        AND(Fact(isue1='slow'), Fact(isue2='noisy'), Fact(isue3='not overheating')),
        AND(Fact(isue1='dim screen'), Fact(isue2='battery swelling'), Fact(isue3='overheating'))
    ))
    def mark_diagnosed(self):
        self.declare(Fact(diagnosed=True))

    #this code block is run if the user inputs slow noisy and overheaing
    @Rule(Fact(isue1='slow'), Fact(isue2='noisy'), Fact(isue3='overheating'))
    def isMalware(self):
        print("You have malware")
        self.declare(Fact(diagnosed=True))  # Declare diagnosed=True after this rule fires

    @Rule(Fact(isue1='slow'), Fact(isue2='noisy'), Fact(isue3='not overheating'))
    def isRAM(self):
        print("You need more RAM")
        self.declare(Fact(diagnosed=True))  # Declare diagnosed=True after this rule fires

    @Rule(Fact(isue1='dim screen'), Fact(isue2='battery swelling'), Fact(isue3='overheating'))
    def isBattery(self):
        print("You need a new battery")
        self.declare(Fact(diagnosed=True))  # Declare diagnosed=True after this rule fires

# i1 = input("enter first input...")
# i2 = input("enter second input...")
# i3 = input("enter third input...")
i1 = 'slow'
i2 = 'noisy'
i3 = 'overheating'

engine = Diagnoser()
engine.reset()
engine.declare(Fact(isue1 = i1), Fact(isue2 = i2), Fact(isue3 = i3))
engine.run()

TypeError: Unknown type <class 'tuple'>

In [1]:
class KnowledgeBase:
    def __init__(self):
        self.diseases = {
            "Common Cold": {"symptoms": ["cough", "sneezing", "sore throat"], "treatment": "rest and hydration"},
            "Flu": {"symptoms": ["fever", "fatigue", "body aches"], "treatment": "antiviral medication"},
            "Pneumonia": {"symptoms": ["chest pain", "cough", "fever"], "treatment": "antibiotics"},
            "Tuberculosis": {"symptoms": ["cough with blood", "fatigue", "weight loss"], "treatment": "antibiotics"},
            "Allergies": {"symptoms": ["sneezing", "itchy eyes", "rash"], "treatment": "antihistamines"}
        }

        self.relationships = {
            "Common Cold": ["Flu", "Allergies"],
            "Flu": ["Common Cold", "Pneumonia"],
            "Pneumonia": ["Flu"],
            "Tuberculosis": [],
            "Allergies": ["Common Cold"],
            "cough": ["Common Cold", "Flu", "Pneumonia", "Tuberculosis"],
            "sneezing": ["Common Cold", "Flu", "Allergies"],
            "fever": ["Flu", "Pneumonia"]
        }

    def diagnose(self, symptoms):
        possible_diseases = {}
        for symptom in symptoms:
            for disease in self.relationships.get(symptom, []):
                possible_diseases[disease] = possible_diseases.get(disease, 0) + 1

        # Sort diseases by the number of matching symptoms (most relevant first)
        sorted_diseases = sorted(possible_diseases.items(), key=lambda x: x[1], reverse=True)

        return [disease for disease, _ in sorted_diseases]

# Interactive mode
kb = KnowledgeBase()
print("Welcome to the Medical Diagnosis Knowledge-Based System")
print("Possible symptoms:", ", ".join(kb.relationships.keys()))
print("\nEnter symptoms (comma separated):")
user_input = input(">").lower().split(",")

symptoms = [symptom.strip() for symptom in user_input]
diagnoses = kb.diagnose(symptoms)

if diagnoses:
    print("Possible diagnoses:")
    for disease in diagnoses:
        print(f"- {disease}: {kb.diseases[disease]['treatment']}")
else:
    print("No diagnoses found. Please consult a doctor.")

Welcome to the Medical Diagnosis Knowledge-Based System
Possible symptoms: Common Cold, Flu, Pneumonia, Tuberculosis, Allergies, cough, sneezing, fever

Enter symptoms (comma separated):
Possible diagnoses:
- Flu: antiviral medication
- Common Cold: rest and hydration
- Pneumonia: antibiotics
- Tuberculosis: antibiotics
- Allergies: antihistamines


In [ ]:
import datetime
import tkinter as tk
from tkinter import messagebox, scrolledtext

class KnowledgeBase:
    def __init__(self):
        self.diseases = {
            "Common Cold": {"symptoms": ["cough", "sneezing", "sore throat"],
                             "treatment": "rest and hydration", "severity": "low"},
            "Flu": {"symptoms": ["fever", "fatigue", "body aches"],
                     "treatment": "antiviral medication", "severity": "medium"},
            "Pneumonia": {"symptoms": ["chest pain", "cough", "fever", "shortness of breath"],
                           "treatment": "antibiotics", "severity": "high"},
            "Tuberculosis": {"symptoms": ["cough with blood", "fatigue", "weight loss", "night sweats"], 
                             "treatment": "antibiotics", "severity": "high"},
            "Allergies": {"symptoms": ["sneezing", "itchy eyes", "rash"], 
                          "treatment": "antihistamines", "severity": "low"},
            "COVID-19": {"symptoms": ["fever", "cough", "shortness of breath", "loss of taste or smell"],
                          "treatment": "quarantine and medical care", "severity": "high"},
            "Bronchitis": {"symptoms": ["cough", "fatigue", "shortness of breath", "chest discomfort"],
                            "treatment": "rest and hydration", "severity": "medium"}
        }

        self.relationships = {
            "Common Cold": ["Flu", "Allergies"],
            "Flu": ["Common Cold", "Pneumonia"],
            "Pneumonia": ["Flu", "COVID-19", "Bronchitis"],
            "Tuberculosis": [],
            "Allergies": ["Common Cold"],
            "COVID-19": ["Pneumonia", "Flu"],
            "Bronchitis": ["Pneumonia", "Flu"],
            "cough": ["Common Cold", "Flu", "Pneumonia", "Tuberculosis", "COVID-19", "Bronchitis"],
            "sneezing": ["Common Cold", "Flu", "Allergies"],
            "fever": ["Flu", "Pneumonia", "COVID-19"],
            "shortness of breath": ["Pneumonia", "COVID-19", "Bronchitis"],
            "loss of taste or smell": ["COVID-19"],
            "chest pain": ["Pneumonia", "Bronchitis"],
            "fatigue": ["Flu", "Tuberculosis", "COVID-19", "Bronchitis"],
            "body aches": ["Flu"],
            "sore throat": ["Common Cold"],
            "itchy eyes": ["Allergies"],
            "rash": ["Allergies"],
            "weight loss": ["Tuberculosis"],
            "night sweats": ["Tuberculosis"]
        }

        self.patient_history = {}

    def diagnose(self, symptoms, patient_id=None):
        possible_diseases = {}
        for symptom in symptoms:
            for disease in self.relationships.get(symptom, []):
                possible_diseases[disease] = possible_diseases.get(disease, 0) + 1

        # Calculate=ing probability
        total_symptoms = len(symptoms)
        disease_probabilities = {}
        for disease, count in possible_diseases.items():
            disease_probabilities[disease] = (count / total_symptoms) * 100

        # Sorting diseases by probability (most relevant first)
        sorted_diseases = sorted(disease_probabilities.items(), key=lambda x: x[1], reverse=True)

        # Tracking patient history
        if patient_id:
            self.track_patient_history(patient_id, symptoms, sorted_diseases)

        return sorted_diseases

    def track_patient_history(self, patient_id, symptoms, diagnoses):
        if patient_id not in self.patient_history:
            self.patient_history[patient_id] = []

        self.patient_history[patient_id].append({
            "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "symptoms": symptoms,
            "diagnoses": diagnoses
        })

    def get_patient_history(self, patient_id):
        return self.patient_history.get(patient_id, [])

    def get_severity(self, disease):
        return self.diseases.get(disease, {}).get("severity", "unknown")


class MedicalDiagnosisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Medical Diagnosis System")
        self.kb = KnowledgeBase()

        # GUI Components
        self.label_symptoms = tk.Label(root, text="Enter Symptoms (comma separated):")
        self.label_symptoms.pack(pady=5)

        self.entry_symptoms = tk.Entry(root, width=50)
        self.entry_symptoms.pack(pady=5)

        self.label_patient_id = tk.Label(root, text="Patient ID (optional):")
        self.label_patient_id.pack(pady=5)

        self.entry_patient_id = tk.Entry(root, width=20)
        self.entry_patient_id.pack(pady=5)

        self.button_diagnose = tk.Button(root, text="Diagnose", command=self.diagnose)
        self.button_diagnose.pack(pady=10)

        self.text_output = scrolledtext.ScrolledText(root, width=80, height=20, wrap=tk.WORD)
        self.text_output.pack(pady=10)

        self.button_history = tk.Button(root, text="View Patient History", command=self.view_history)
        self.button_history.pack(pady=10)

    def diagnose(self):
        symptoms = self.entry_symptoms.get().strip().lower().split(",")
        symptoms = [symptom.strip() for symptom in symptoms]
        patient_id = self.entry_patient_id.get().strip() or None

        if not symptoms:
            messagebox.showwarning("Input Error", "Please enter at least one symptom.")
            return

        diagnoses = self.kb.diagnose(symptoms, patient_id)

        self.text_output.delete(1.0, tk.END)
        if diagnoses:
            self.text_output.insert(tk.END, "Possible Diagnoses (with probabilities):\n")
            for disease, probability in diagnoses:
                severity = self.kb.get_severity(disease)
                self.text_output.insert(tk.END, f"- {disease}: {probability:.2f}% (Severity: {severity})\n")
                self.text_output.insert(tk.END, f"  Treatment: {self.kb.diseases[disease]['treatment']}\n\n")
        else:
            self.text_output.insert(tk.END, "No diagnoses found. Please consult a doctor.")

    def view_history(self):
        patient_id = self.entry_patient_id.get().strip()
        if not patient_id:
            messagebox.showwarning("Input Error", "Please enter a Patient ID to view history.")
            return

        history = self.kb.get_patient_history(patient_id)
        self.text_output.delete(1.0, tk.END)

        if history:
            self.text_output.insert(tk.END, f"Patient History for ID: {patient_id}\n")
            for entry in history:
                self.text_output.insert(tk.END, f"\nDate: {entry['date']}\n")
                self.text_output.insert(tk.END, f"Symptoms: {', '.join(entry['symptoms'])}\n")
                self.text_output.insert(tk.END, "Diagnoses:\n")
                for disease, probability in entry['diagnoses']:
                    self.text_output.insert(tk.END, f"- {disease}: {probability:.2f}%\n")
        else:
            self.text_output.insert(tk.END, f"No history found for Patient ID: {patient_id}")


In [ ]:
# Running the application
if __name__ == "__main__":
    root = tk.Tk()
    app = MedicalDiagnosisApp(root)
    root.mainloop()